# config file into solara web app

In [1]:
import solara
from ipyleaflet import Map, DrawControl, basemaps
from ipywidgets import Output, Dropdown, FloatSlider, Text
import json



In [2]:
# HERE: DROP-DOWN MENU; people won't have datasets on their local drive

FA_databases = ["Scheveningen", "Option B", "Option C"] #################################
FA_database = solara.reactive("Scheveningen")


@solara.component
def Page():
    solara.Select(label="FloodAdapt data base", value=FA_database, values=FA_databases)
    
    solara.Markdown(f"**Selected**: {FA_database.value}")


########################################################################################################################################################################################
# The following line is required only when running the code in a Jupyter notebook:
Page()

Cannot show ipywidgets in text

In [3]:
if FA_database.value == "Scheveningen":
    fa_database = Path(r"/mnt/c/Users/santjer/OneDrive - Stichting Deltares/Documents/DestinE/Technical/01_FloodAdapt/scheveningen")
elif FA_database.value == "Option B":
    fa_database = Path(r"/mnt/c/Users/santjer/OneDrive - Stichting Deltares/Documents/DestinE/Technical/01_FloodAdapt/OPTIONB/") #########################
else:
    fa_database = Path(r"/mnt/c/Users/santjer/OneDrive - Stichting Deltares/Documents/DestinE/Technical/01_FloodAdapt/OPTIONC/") #########################


# HERE: whatever the solutoin is; people won't have datasets on their local drive

# Path to data folder. If (relative) file paths are used when specifying data sets, it will be looked for in this folder.
# data_folder = Path("/home/wotromp/InterTwin/Data")  


# OPTIONAL: HydroMT data catalogue from which to fetch data. The order matters here, as datasets are looked for in a catalogue in order from first catalogue name in list to last.
data_catalogues = [Path(r"/mnt/c/Users/santjer/OneDrive - Stichting Deltares/Documents/DestinE/Technical/01_FloodAdapt/deltares_data_wsl.yml")]

NameError: name 'Path' is not defined

# Page map and tabs

In [7]:


# Reactive variables to store drawn polygons and their coordinates
polygon_coordinates = solara.reactive([])
drawn_polygons = solara.reactive([])
selected_tab = solara.reactive('Tab 1')  # Track the selected tab

# Function to handle drawing events
def handle_draw(target, action, geo_json):
    print(f"Action: {action}, GeoJSON: {geo_json}")  # Debugging: Check incoming data
    if action == 'created' and 'geometry' in geo_json:
        if geo_json['geometry']['type'] == 'Polygon':
            coords = geo_json['geometry']['coordinates'][0]
            polygon_coordinates.value = coords
            drawn_polygons.value.append({'type': 'Polygon', 'coordinates': coords})
            print("Polygon coordinates updated:", polygon_coordinates.value)
            print("All drawn polygons updated:", drawn_polygons.value)

# Initialize the map
m = Map(center=(52.08654741528378, 4.295223531699989), zoom=10, scroll_wheel_zoom=True, basemap=basemaps.OpenStreetMap.Mapnik)

# Add draw control to the map
draw_control = DrawControl(
    polyline={'shapeOptions': {'color': 'blue', 'weight': 4}},
    polygon={'shapeOptions': {'color': 'red', 'weight': 4}},
    rectangle={},
    circle={},
    marker={}
)
draw_control.on_draw(handle_draw)
m.add_control(draw_control)

# Create an Output widget to display the map
map_output = Output()
with map_output:
    display(m)



# Define UI components for tabs
@solara.component
def Tab1():
    with solara.Card("Weather Event"):
        # solara.Text("Configure the weather Event:")
        # dropdown = Dropdown(options=['Option 1', 'Option 2', 'Option 3'], value='Option 1')
        # solara.Card(dropdown)  # Directly include the widget in the Card

        # solara.Text("Slider:")
        # slider = FloatSlider(value=50, min=0, max=100, step=1)
        # solara.Card(slider)  # Directly include the widget in the Card

        # solara.Text("Text Input:")
        # text_input = Text(value='Type here')
        # solara.Card(text_input)  # Directly include the widget in the Card
        event_name_input = solara.reactive("Event name") 
        solara.Text("Configure the weather Event:")
        solara.InputText("Name of the event", value=event_name_input)
        solara.Markdown(f"**Your name of the event**: {event_name_input.value}")

@solara.component
def Tab2():
    with solara.Card("Projections"):
        solara.Text("Another Dropdown Menu:")
        dropdown2 = Dropdown(options=['Option A', 'Option B', 'Option C'], value='Option A')
        solara.Card(dropdown2)  # Directly include the widget in the Card

        solara.Text("Another Slider:")
        slider2 = FloatSlider(value=25, min=0, max=50, step=0.5)
        solara.Card(slider2)  # Directly include the widget in the Card

        solara.Text("Another Text Input:")
        text_input2 = Text(value='Enter text')
        solara.Card(text_input2)  # Directly include the widget in the Card

@solara.component
def Tab3():
    with solara.Card("Measures"):
        solara.Text("Yet Another Dropdown Menu:")
        dropdown3 = Dropdown(options=['Choice X', 'Choice Y', 'Choice Z'], value='Choice X')
        solara.Card(dropdown3)  # Directly include the widget in the Card

        solara.Text("Yet Another Slider:")
        slider3 = FloatSlider(value=75, min=0, max=150, step=5)
        solara.Card(slider3)  # Directly include the widget in the Card

        solara.Text("Yet Another Text Input:")
        text_input3 = Text(value='Write here')
        solara.Card(text_input3)  # Directly include the widget in the Card

@solara.component
def SettingsTabs():
    with solara.Row():
        solara.Button("Tab 1", on_click=lambda: selected_tab.set('Tab 1'))
        solara.Button("Tab 2", on_click=lambda: selected_tab.set('Tab 2'))
        solara.Button("Tab 3", on_click=lambda: selected_tab.set('Tab 3'))

    if selected_tab.value == 'Tab 1':
        Tab1()
    elif selected_tab.value == 'Tab 2':
        Tab2()
    elif selected_tab.value == 'Tab 3':
        Tab3()

@solara.component
def App():
    # Layout with columns
    with solara.Columns():
        with solara.Column(style={"width": "70%", "min-width": "500px"}):
            solara.use_memo(lambda: display(m), [])
        with solara.Column(style={"width": "30%", "min-width": "200px"}):
            SettingsTabs()

# Run the Solara app
App()

Cannot show ipywidgets in text

In [31]:
drawn_polygons.value


[{'type': 'Polygon',
  'coordinates': [[4.198425, 52.053166],
   [4.24649, 52.010077],
   [4.37146, 52.036273],
   [4.326141, 52.051477],
   [4.272583, 52.079337],
   [4.198425, 52.053166]]}]